In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from lightgbm.sklearn import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

In [2]:
DIR = "../../data/"
SMOOTHIE = "Smoothie King/"

smoothie_demographic = pd.read_csv(DIR + SMOOTHIE + "processed_demographic.csv")
smoothie_stores = pd.read_csv(DIR + SMOOTHIE + "smoothie_king_stores.csv")
smoothie_poi_variables = pd.read_csv(DIR + SMOOTHIE + "processed_poi.csv")
# smoothie_sister = pd.read_csv(DIR + SMOOTHIE + "competition_sister_variables.csv")
smoothie_trade_area = pd.read_csv(DIR + SMOOTHIE + "processed_trade_area.csv").rename(columns={"store_num": "store"})

In [3]:
smoothie_merged = smoothie_stores.merge(
    smoothie_demographic, on="store", how="outer"
).merge(
    smoothie_poi_variables, on="store", how="outer"
).merge(
    smoothie_trade_area, on="store", how="outer"
)
smoothie_merged

,store,longitude,latitude,category,cbsa_name,dma_name,state_name,market_size,store_density,age0018_p_10mi,...,popgrfy_ta,popgrpy_ta,poverty_inpoverty_p_ta,spend_breakfastbrunch_ta,spend_dinner_ta,spend_foodbev_ta,spend_lunch_ta,wealth_hhavg_ta,wealth_hhtotal_ta,white_p_ta
0,SK 1504,-97.650392,30.519353,SHOPPING,"Austin-Round Rock, TX","Austin, TX",Texas,Large Metro (2),Light Suburban,0.2805,...,8.3789,3.9235,0.0611,7069439.0,40790484.0,230383651.0,23166216.0,240573.0,25223.0,0.4897
1,SK 0057,-88.171150,30.672501,SHOPPING,"Mobile, AL","Mobile et al, AL-FL",Alabama,Medium City (4),Light Suburban,0.2264,...,0.6017,0.3932,0.1830,4724526.0,25460067.0,160135521.0,14653701.0,217054.0,22216.0,0.5129
2,SK 1415,-90.535722,38.784250,HOME,"St. Louis, MO-IL","St. Louis, MO",Missouri,Very Large Metro (1),Light Suburban,0.2129,...,2.5003,0.7142,0.0639,4501211.0,24794631.0,151609187.0,14129014.0,245860.0,19907.0,0.8459
3,SK 1231,-80.134700,26.100737,TRAVEL,"Miami-Fort Lauderdale et al, FL","Miami-Ft. Lauderdale, FL",Florida,Very Large Metro (1),Suburban,0.2174,...,7.6482,12.7188,0.0831,4038906.0,23214366.0,146417939.0,12859709.0,255812.0,22124.0,0.7020
4,SK 1535,-96.856651,32.996408,WORK,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Light Suburban,0.2620,...,2.1871,-5.7794,0.0800,5919218.0,32751951.0,207616741.0,18696840.0,216763.0,29251.0,0.4593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,SK 1536,-96.872596,32.647809,HOME,"Dallas-Fort Worth-Arlington, TX","Dallas-Ft. Worth, TX",Texas,Very Large Metro (1),Exurban,0.2664,...,2.0540,-0.2994,0.1833,7614977.0,40037412.0,264008408.0,23332500.0,199437.0,29430.0,0.1490
792,SK 1886,-105.077634,40.564695,OTHER,"Fort Collins, CO","Denver, CO",Colorado,Medium City (4),Light Suburban,0.2224,...,4.7691,2.3442,0.2202,5118356.0,28458285.0,197532602.0,16637181.0,216659.0,25712.0,0.7874
793,SK 0162,-95.478001,30.316531,SHOPPING,"Houston-The Woodlands et al, TX","Houston, TX",Texas,Very Large Metro (1),Exurban,0.2412,...,9.9822,8.1691,0.1202,4677280.0,25445615.0,161088758.0,14627532.0,216855.0,19407.0,0.5041
794,SK 1449,-78.968258,35.064994,SHOPPING,"Fayetteville, NC","Raleigh et al, NC",North Carolina,Medium City (4),Exurban,0.2614,...,-0.3816,-1.0761,0.1809,10318478.0,54577513.0,349542567.0,31683602.0,208448.0,41681.0,0.3933


In [4]:
train_df, test_df = train_test_split(smoothie_merged, test_size=0.1, random_state=42)
X_train = train_df.drop(columns=["category"])
y_train = train_df["category"]
X_test = test_df.drop(columns=["category"])
y_test = test_df["category"]

In [5]:
drop_features = ['store',
 'longitude',
 'latitude',
 'cbsa_name',
 'dma_name',
 'state_name',
 ]

passthrough_features = []

ordinal_features_oth = [
    "market_size",
    "store_density",
]
ordering_ordinal_oth = [
    ["Very Large Metro (1)", "Large Metro (2)", "Large City (3)", "Medium City (4)", "Small City (5)", "Small Town (6)"],
    ["Rural", "Exurban", "Suburban", "Light Suburban", "Light Urban", "Urban", "Super Urban"],
]
numeric_features = list(set(smoothie_merged.select_dtypes(include=np.number).columns.tolist()) - {"longitude", "latitude"})

In [6]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy="median"), 
    StandardScaler()
)

ordinal_transformer_oth = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder(categories=ordering_ordinal_oth),
)

preprocessor = make_column_transformer(
    ("drop", drop_features),
    (numeric_transformer, numeric_features),
    (ordinal_transformer_oth, ordinal_features_oth),
    ("passthrough", passthrough_features),
)

In [7]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('drop', 'drop',
                                 ['store', 'longitude', 'latitude', 'cbsa_name',
                                  'dma_name', 'state_name']),
                                ('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['gq_other_p_3mi', 'emp_military_10mi',
                                  'hh_2pers_p_10mi', 'hu_vacant_5mi',
                                  'com0205_p_1mi', 'other_p_ta', 'genx_p_10mi'...
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Very '
                                                                              'Large '
                                                                              'Metro '
                                                                              '(1)',
                                                                              'Large '
                                                                              'Metro '
                                                                              '(2)',
                                                                              'Large '
                                                                              'City '
                                                                              '(3)',
                                                                              'Medium '
                                                                              'City '
                                                                              '(4)',
                                                                              'Small '
                                                                              'City '
                                                                              '(5)',
                                                                              'Small '
                                                                              'Town '
                                                                              '(6)'],
                                                                             ['Rural',
                                                                              'Exurban',
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Suburban',
                                                                              'Light '
                                                                              'Urban',
                                                                              'Urban',
                                                                              'Super '
                                                                              'Urban']]))]),
                                 ['market_size', 'store_density'])])

In [8]:
column_names = (
    numeric_features
    + preprocessor.named_transformers_['pipeline-2']['ordinalencoder'].get_feature_names_out().tolist()
)
len(column_names)

920

In [9]:
transformed_X_train = preprocessor.transform(X_train)

In [10]:
transformed_X_train_df = pd.DataFrame(transformed_X_train, columns=column_names)
transformed_X_train_df

,gq_other_p_3mi,emp_military_10mi,hh_2pers_p_10mi,hu_vacant_5mi,com0205_p_1mi,other_p_ta,genx_p_10mi,com0002_p_10mi,emp_retail_trade_p_5mi,emp_educ_services_p_10mi,...,dtpop_students_p_ta,pop_migration_10mi,com12pl_p_ta,hh_type_male_child_p_5mi,medsalcy_ta,occhu_2mi,percapita_inc_3mi,asian_p_3mi,x0,x1
0,3.531649,-0.260143,0.069611,-0.619462,-0.823084,0.261417,1.121923,-0.775356,-0.180202,0.170414,...,0.589739,1.036752,0.864670,0.747918,0.784573,-0.479087,0.486845,-0.354294,1.0,0.0
1,0.045627,-0.071340,-0.347581,0.276204,0.286615,-0.034546,-0.212734,-0.425355,-0.694044,-0.915408,...,-0.755550,0.433997,-0.303647,-0.924081,0.730122,0.872324,1.025312,-0.095207,0.0,3.0
2,2.314058,-0.260143,-1.344205,-0.616595,-1.608248,-0.004950,0.133288,-1.633164,-1.089913,0.913859,...,0.139076,1.261008,2.269661,-1.324808,0.035313,-0.156323,0.652047,4.134616,0.0,1.0
3,-0.068669,-0.228676,-1.662066,-0.414359,-0.593641,-0.315711,-0.181839,-1.449059,-0.796288,0.030203,...,2.698979,-0.575687,1.114112,0.540646,1.531321,-0.012966,-0.436286,-0.594188,0.0,3.0
4,0.531384,-0.139996,-0.675374,-0.547670,-0.167035,-0.789252,-0.880063,0.503262,-0.961452,0.574744,...,-0.494199,0.202209,0.396340,-0.606263,0.315890,-0.372332,1.044339,0.048729,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,0.377426,-0.260143,-0.291293,1.045726,0.445393,-0.278716,1.702746,0.072335,-1.441213,-1.084965,...,0.802506,0.534032,-1.148648,-0.633899,0.683691,1.258779,2.252722,-0.446413,0.0,2.0
712,-0.665310,0.226167,-0.086009,-0.465605,0.249102,-0.559880,-0.039723,0.199793,1.128000,-0.762153,...,-0.100496,-0.173001,-0.718167,0.167555,-1.458898,-0.360386,0.134349,0.327007,3.0,1.0
713,0.855080,-0.260143,-0.874037,-0.370759,-1.548052,0.305811,1.912831,-1.716112,0.278586,1.491007,...,2.129367,1.656586,2.385033,-0.067354,0.817108,-0.236059,0.121644,-0.310153,0.0,1.0
714,-0.272951,-0.260143,-0.314471,-0.247602,1.863051,-1.551356,-0.138587,-0.018705,-1.328482,0.189978,...,-0.515978,-0.146693,-1.266301,-0.550990,-0.703299,-0.274677,-0.631956,-0.822568,2.0,1.0


In [11]:
import optuna
from optuna.integration import LightGBMPruningCallback

In [12]:
best_booster = None
lgbm = None

In [13]:
def objective(trial, X, y):
    global lgbm
    param_grid = {
        #         "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [200]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 400, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 500, step=100),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        # print(idx)
        # print(train_idx)
        # print(test_idx)
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        # print('X_train shape: ', X_train.shape)
        # print('X_test shape: ', X_test.shape)
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        # print('y_train shape: ', y_train.shape)
        # print('y_test shape: ', y_test.shape)

        lgbm = LGBMClassifier(objective="multiclass", **param_grid)
        lgbm.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="multi_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "multi_logloss")
            ],  # Add a pruning callback
        )
        preds = lgbm.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [14]:
def callback(study, trial):
    global best_booster
    if study.best_trial == trial:
        best_booster = lgbm

In [15]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, transformed_X_train_df, y_train)
study.optimize(func, n_trials=100, callbacks=[callback])

[I 2023-05-17 11:54:17,549] A new study created in memory with name: LGBM Classifier


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=14.849322381913252, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=14.849322381913252
[LightGBM] [Warning] lambda_l1 is set=95, reg_alpha=0.0 will be ignored. Current value: lambda_l1=95
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_sa

[I 2023-05-17 11:54:17,955] Trial 0 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.0414085392272303, 'num_leaves': 180, 'max_depth': 3, 'min_data_in_leaf': 300, 'max_bin': 210, 'lambda_l1': 95, 'lambda_l2': 85, 'min_gain_to_split': 14.849322381913252, 'bagging_fraction': 0.2, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 0 with value: 1.4718720941213552.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=300
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=14.849322381913252, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=14.849322381913252
[LightGBM] [Warning] lambda_l1 is set=95, reg_alpha=0.0 will be ignored. Current value: lambda_l1=95
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=300, min_child_sa

[I 2023-05-17 11:54:18,251] Trial 1 finished with value: 1.4718720941213552 and parameters: {'n_estimators': 200, 'learning_rate': 0.030150295512012225, 'num_leaves': 240, 'max_depth': 5, 'min_data_in_leaf': 500, 'max_bin': 231, 'lambda_l1': 15, 'lambda_l2': 70, 'min_gain_to_split': 5.4202395139428745, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 1.4718720941213552.


...

[I 2023-05-17 11:55:19,288] Trial 96 pruned. Trial was pruned at iteration 101.
[I 2023-05-17 11:55:19,350] Trial 97 pruned. Trial was pruned at iteration 0.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=50, reg_lambda=0.0 will be ignored. Current value: lambda_l2=50
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=2.230443979234765, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.230443979234765
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samp

[I 2023-05-17 11:55:21,048] Trial 98 finished with value: 1.4324071573871202 and parameters: {'n_estimators': 200, 'learning_rate': 0.054649202445685975, 'num_leaves': 220, 'max_depth': 4, 'min_data_in_leaf': 200, 'max_bin': 244, 'lambda_l1': 20, 'lambda_l2': 50, 'min_gain_to_split': 2.230443979234765, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 98 with value: 1.4324071573871202.


[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=2.1627231830949416, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.1627231830949416
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_sa

[I 2023-05-17 11:55:22,742] Trial 99 finished with value: 1.430884803889265 and parameters: {'n_estimators': 200, 'learning_rate': 0.0542703507698215, 'num_leaves': 240, 'max_depth': 4, 'min_data_in_leaf': 200, 'max_bin': 241, 'lambda_l1': 20, 'lambda_l2': 45, 'min_gain_to_split': 2.1627231830949416, 'bagging_fraction': 0.7, 'bagging_freq': 1, 'feature_fraction': 0.4}. Best is trial 99 with value: 1.430884803889265.


In [16]:
print(f"\tBest value (multi_logloss): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (multi_logloss): 1.43088
	Best params:
		n_estimators: 200
		learning_rate: 0.0542703507698215
		num_leaves: 240
		max_depth: 4
		min_data_in_leaf: 200
		max_bin: 241
		lambda_l1: 20
		lambda_l2: 45
		min_gain_to_split: 2.1627231830949416
		bagging_fraction: 0.7
		bagging_freq: 1
		feature_fraction: 0.4


In [17]:
best_booster

LGBMClassifier(bagging_fraction=0.7, bagging_freq=1, feature_fraction=0.4,
               lambda_l1=20, lambda_l2=45, learning_rate=0.0542703507698215,
               max_bin=241, max_depth=4, min_data_in_leaf=200,
               min_gain_to_split=2.1627231830949416, n_estimators=200,
               num_leaves=240, objective='multiclass')

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
smoothie_merged["category"] = le.fit_transform(smoothie_merged["category"])

In [20]:
class_weight = {
    "HOME": 0.31,
    "OTHER": 0.13,
    "SHOPPING": 0.22,
    "TRAVEL": 0.15,
    "WORK": 0.28
}
encoded_class_weights = {i: class_weight[label] for i, label in enumerate(le.classes_)}
class_weight = encoded_class_weights

In [21]:
pipe_rf = make_pipeline(
    preprocessor,
    RandomForestClassifier(n_jobs=-1, random_state=42)
)
pipe_rf.fit(X_train, y_train);

In [22]:
rf_param_grid = {
    "randomforestclassifier__n_estimators": [25, 50, 100, 150],
    "randomforestclassifier__max_features": ["sqrt", "log2", None],
    "randomforestclassifier__max_depth": [5, 10, 20, 50],
    "randomforestclassifier__max_leaf_nodes": [30, 50, 70],
    "randomforestclassifier__class_weight": ["balanced", [class_weight]]
}
rf_grid_search = GridSearchCV(
    pipe_rf, rf_param_grid, cv=5, n_jobs=-1, return_train_score=True
)
rf_grid_search.fit(X_train, y_train);

In [23]:
rf_tuned = rf_grid_search.best_estimator_
best_rf = rf_tuned.named_steps["randomforestclassifier"]
type(best_rf)
best_rf

RandomForestClassifier(class_weight='balanced', max_depth=20, max_features=None,
                       max_leaf_nodes=50, n_jobs=-1, random_state=42)

In [24]:
def evaluate_models(X_train, y_train,
		            X_test, y_test,
		    		rfe_estimator, 
		    		rfe_num_features_start, rfe_num_features_end, rfe_num_features_step, 
				    pipe_model):
	
	for i in range(rfe_num_features_start, rfe_num_features_end+1, rfe_num_features_step):
		print('---- RFE with ', i, ' features selected ----')
		
		rfe = RFE(estimator=rfe_estimator, n_features_to_select=i)
		pipe_rfe_model = make_pipeline(preprocessor, 
			                           rfe, 
				                       pipe_model)
		
		# print(pipe_rfe_model)
		
		pipe_rfe_model.fit(X_train, pd.DataFrame(y_train))
		
		print(pipe_rfe_model.score(X_train, pd.DataFrame(y_train)))
		print(pipe_rfe_model.score(X_test, pd.DataFrame(y_test)))	

In [25]:
evaluate_models(X_train, y_train,
		        X_test, y_test,
		    	best_booster, 
		    	8, 25, 1, 
				rf_tuned)

---- RFE with  8  features selected ----
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] lambda_l2 is set=45, reg_lambda=0.0 will be ignored. Current value: lambda_l2=45
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] min_gain_to_split is set=2.1627231830949416, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=2.1627231830949416
[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20


ValueError: Specifying the columns using strings is only supported for pandas DataFrames